# Russian Troll Tweets: Data Cleaning

**EDA Ideas** 
- want to see if rate of tweets accelerates as the election comes up
    - bar graph of # tweets per month in this dataset
    - same as above but for a single user (maybe plot 3-5 together?)
- same idea as above but for use of hashtags 
    - what is the rate of their use? 
    - a racing bar chart would be cool here (see [Lucy's resource](https://observablehq.com/@d3/gallery))

In [1]:
# Basics
from collections import Counter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import re
import string
import datetime
import pickle 

# visualizations
from wordcloud import WordCloud
from nltk import FreqDist

import nltk
from nltk.tokenize import word_tokenize, sent_tokenize, MWETokenizer
from nltk.util import ngrams
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem.porter import PorterStemmer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.corpus import words
from nltk.corpus import wordnet

import spacy

from preprocessing_funcs import clean_tweet, get_hashtags

In [ ]:
# nltk.download("punkt")
# nltk.download("stopwords")
# nltk.download("words")
# nltk.download("europarl_raw")
# nltk.download("wordnet")
# nltk.download("averaged_perceptron_tagger")

In [2]:
raw_df = pd.read_csv('../data_files/tweets.csv.zip')

In [3]:
raw_df.head()

,user_id,user_key,created_at,created_str,retweet_count,retweeted,favorite_count,text,tweet_id,source,hashtags,expanded_urls,posted,mentions,retweeted_status_id,in_reply_to_status_id
0,1.868981e+09,ryanmaxwell_1,1.458672e+12,2016-03-22 18:31:42,NaN,NaN,NaN,#IslamKills Are you trying to say that there w...,7.123460e+17,NaN,"[""IslamKills""]",[],POSTED,[],NaN,NaN
1,2.571870e+09,detroitdailynew,1.476133e+12,2016-10-10 20:57:00,0.0,False,0.0,"Clinton: Trump should’ve apologized more, atta...",7.855849e+17,"<a href=""http://twitterfeed.com"" rel=""nofollow...",[],"[""http://detne.ws/2e172jF""]",POSTED,[],NaN,NaN
2,1.710805e+09,cookncooks,1.487767e+12,2017-02-22 12:43:43,NaN,NaN,NaN,RT @ltapoll: Who was/is the best president of ...,8.343832e+17,NaN,[],[],POSTED,[],NaN,NaN
3,2.584153e+09,queenofthewo,1.482765e+12,2016-12-26 15:06:41,NaN,NaN,NaN,RT @jww372: I don't have to guess your religio...,8.134006e+17,NaN,"[""ChristmasAftermath""]",[],POSTED,[],NaN,NaN
4,1.768260e+09,mrclydepratt,1.501987e+12,2017-08-06 02:36:24,NaN,NaN,NaN,RT @Shareblue: Pence and his lawyers decided w...,8.940243e+17,NaN,[],[],POSTED,[],NaN,NaN


In [4]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203482 entries, 0 to 203481
Data columns (total 16 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   user_id                195417 non-null  float64
 1   user_key               203482 non-null  object 
 2   created_at             203461 non-null  float64
 3   created_str            203461 non-null  object 
 4   retweet_count          58083 non-null   float64
 5   retweeted              58083 non-null   object 
 6   favorite_count         58083 non-null   float64
 7   text                   203461 non-null  object 
 8   tweet_id               201168 non-null  float64
 9   source                 58084 non-null   object 
 10  hashtags               203482 non-null  object 
 11  expanded_urls          203482 non-null  object 
 12  posted                 203482 non-null  object 
 13  mentions               203482 non-null  object 
 14  retweeted_status_id    39651 non-nul

In [5]:
# Get date information
raw_df['date'] = pd.to_datetime(raw_df['created_str']).dt.date

start_date = min(raw_df.date)
end_date = max(raw_df.date)
time_delta_years = (max(raw_df.date) - min(raw_df.date)).days / 365

print(f'Data date range: {start_date} to {end_date}')
print(f'Time period: {time_delta_years:.2f} years')

Data date range: 2014-07-14 to 2017-09-26
Time period: 3.21 years


In [6]:
num_users = len(raw_df.user_key.unique())
num_tweets = len(raw_df)

print(f'Number of unique users: {num_users}')
print(f'Number of Tweets: {num_tweets}\n')
print(f'Average Tweets per User: {num_tweets/num_users:.2f}')
print(f'Average Tweet per User per Day: {(num_tweets/num_users)/(time_delta_years*365):.2f}')
print(f'Average Tweet per User per Week: {(num_tweets/num_users)/(time_delta_years*52):.2f}')
print(f'Average Tweet per User per Month: {(num_tweets/num_users)/(time_delta_years*12):.2f}')
print(f'Average Tweet per User per Year: {(num_tweets/num_users)/(time_delta_years):.2f}')

Number of unique users: 454
Number of Tweets: 203482

Average Tweets per User: 448.20
Average Tweet per User per Day: 0.38
Average Tweet per User per Week: 2.69
Average Tweet per User per Month: 11.65
Average Tweet per User per Year: 139.82


# Just *CLEAN* Tweets!

In [54]:
df = raw_df[['text']].astype(str)

In [55]:
df.head()

,text
0,#IslamKills Are you trying to say that there w...
1,"Clinton: Trump should’ve apologized more, atta..."
2,RT @ltapoll: Who was/is the best president of ...
3,RT @jww372: I don't have to guess your religio...
4,RT @Shareblue: Pence and his lawyers decided w...


In [ ]:
#df['hashtags'] = df['text'].map(get_hashtags)

In [56]:
df['clean'] = df['text'].map(clean_tweet)

In [57]:
df.head()

,text,clean
0,#IslamKills Are you trying to say that there w...,islamkills are you trying to say that there we...
1,"Clinton: Trump should’ve apologized more, atta...",clinton trump should ve apologized more attack...
2,RT @ltapoll: Who was/is the best president of ...,who was is the best president of the past year...
3,RT @jww372: I don't have to guess your religio...,i don t have to guess your religion christmasa...
4,RT @Shareblue: Pence and his lawyers decided w...,pence and his lawyers decided which of his off...


## Remove German Words

In [58]:
# a list of all english words known to nltk
english_words = list(nltk.corpus.words.words())
english_words = [word.lower() for word in english_words]
len(english_words), type(english_words)

(236736, list)

In [59]:
word_net = list(nltk.corpus.wordnet.words())
word_net = [word.lower() for word in word_net]
len(word_net), type(word_net)

(147306, list)

In [60]:
many_words = set(english_words + word_net)

In [61]:
len(many_words), type(many_words)

(323592, set)

In [62]:
'email' in many_words

True

In [63]:
from nltk.corpus.europarl_raw import german

In [64]:
german = list(german.words())
german = [word.lower() for word in german]
len(german), type(german)

(556226, list)

In [65]:
remove_german = set([word for word in german if word not in many_words])

In [66]:
len(remove_german)

28154

In [50]:
remove_german_words = lambda x: " ".join(w for w in nltk.word_tokenize(x) if w not in remove_german)

In [67]:
# drop non-english words
df['clean'] = df['clean'].apply(remove_german_words)

In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203482 entries, 0 to 203481
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    203482 non-null  object
 1   clean   203482 non-null  object
dtypes: object(2)
memory usage: 3.1+ MB


In [69]:
df.head()

,text,clean
0,#IslamKills Are you trying to say that there w...,islamkills are you trying to say that there we...
1,"Clinton: Trump should’ve apologized more, atta...",clinton trump should ve apologized more attack...
2,RT @ltapoll: Who was/is the best president of ...,who was is the best president of the past year...
3,RT @jww372: I don't have to guess your religio...,i don t have to guess your religion christmasa...
4,RT @Shareblue: Pence and his lawyers decided w...,pence and his lawyers decided which of his off...


## Removing stop words

In [72]:
standard_stop_words = stopwords.words("english")
print(standard_stop_words)
type(standard_stop_words), len(standard_stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

(list, 179)

In [73]:
with open('../data_files/twitter_stopwords.txt') as f:
    words = f.read().split(',')
    twitter_stopwords = list(words)

len(twitter_stopwords)    

623

In [74]:
twitter_stopwords = [word.lower() for word in twitter_stopwords]

In [75]:
other_words = ['amp', '…']

In [76]:
all_the_stops = set(standard_stop_words + twitter_stopwords + other_words)

In [77]:
len(all_the_stops)

616

In [78]:
remove_stop_words = lambda x: " ".join(w for w in nltk.word_tokenize(x) if w not in all_the_stops)

In [79]:
# drop stop-words
df['no_stops'] = df['clean'].apply(remove_stop_words)

In [81]:
punctuations = '''!’()-![]{};:+'""\,<>./?@#$%^&*_~'''                                   
no_punc = lambda x: " ".join(w for w in nltk.word_tokenize(x) if w not in punctuations)

In [82]:
df['no_stops'] = df['no_stops'].apply(no_punc)

In [83]:
df.head()

,text,clean,no_stops
0,#IslamKills Are you trying to say that there w...,islamkills are you trying to say that there we...,islamkills trying say terrorist attacks europe...
1,"Clinton: Trump should’ve apologized more, atta...",clinton trump should ve apologized more attack...,clinton trump apologized attacked less
2,RT @ltapoll: Who was/is the best president of ...,who was is the best president of the past year...,best president past retweet
3,RT @jww372: I don't have to guess your religio...,i don t have to guess your religion christmasa...,guess religion christmasaftermath
4,RT @Shareblue: Pence and his lawyers decided w...,pence and his lawyers decided which of his off...,pence lawyers decided official emails public c...


## spaCy

In [84]:
nlp = spacy.load('en_core_web_sm', disable=["parser", "ner"])

In [85]:
spacy_lemmatize = lambda x: " ".join([token.lemma_ for token in nlp(x)])

In [86]:
df["lem"] = df["no_stops"].apply(spacy_lemmatize)

In [87]:
df.head()

,text,clean,no_stops,lem
0,#IslamKills Are you trying to say that there w...,islamkills are you trying to say that there we...,islamkills trying say terrorist attacks europe...,islamkill try say terrorist attack europe refu...
1,"Clinton: Trump should’ve apologized more, atta...",clinton trump should ve apologized more attack...,clinton trump apologized attacked less,clinton trump apologize attack less
2,RT @ltapoll: Who was/is the best president of ...,who was is the best president of the past year...,best president past retweet,good president past retweet
3,RT @jww372: I don't have to guess your religio...,i don t have to guess your religion christmasa...,guess religion christmasaftermath,guess religion christmasaftermath
4,RT @Shareblue: Pence and his lawyers decided w...,pence and his lawyers decided which of his off...,pence lawyers decided official emails public c...,pence lawyer decide official email public coul...


In [88]:
for i in range(0, 5):
    print(df.text[i])
    print(df.lem[i], '\n')

#IslamKills Are you trying to say that there were no terrorist attacks in Europe before refugees were let in?
islamkill try say terrorist attack europe refugee let 

Clinton: Trump should’ve apologized more, attacked less https://t.co/eJampkoHFZ
clinton trump apologize attack less 

RT @ltapoll: Who was/is the best president of the past 25 years? (Vote &amp; Retweet)
good president past retweet 

RT @jww372: I don't have to guess your religion! #ChristmasAftermath
guess religion christmasaftermath 

RT @Shareblue: Pence and his lawyers decided which of his official emails the public could see

https://t.co/HjhPguBK1Y by @alisonrose711
pence lawyer decide official email public could see 



In [89]:
all_words = []
for tweet in list(df['lem']):
    words = tweet.split()
    for word in words:
        all_words.append(word.lower())
    
Counter(all_words).most_common(10)

[('trump', 30887),
 ('clinton', 13015),
 ('hillary', 11780),
 ('obama', 9790),
 ('get', 8862),
 ('say', 8193),
 ('people', 6845),
 ('go', 6262),
 ('make', 6113),
 ('like', 5796)]

## Vectorization

In [90]:
cv = CountVectorizer()
doc_word = cv.fit_transform(df["lem"])
vect = pd.DataFrame(doc_word.toarray(),columns=cv.get_feature_names())
vect

,aa,aaa,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaannnnnnnnnnnnnnnnnnnnnnnnnnnnnnndddddddddddddddddddddddddddddddddd,aaaaaaaaaaaaaaaaaaaaannnnnnnnnnnnnnnnnnnnddddddddddddddddddddddddddddddd,aaaaaaaaannnnnnnnnnnddddddddddddd,aaaaaaaaassssss,aaaaaand,aaaaaayyyuuuummmmmm,aaaaand,aaaand,...,zynalturist,zynischen,zyoritv,zzcrane,zzion,zzjwmc,zzzs,zzzzzz,zzzzzzz,zzzzzzzzzzzzzzzzz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203477,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
203478,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
203479,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
203480,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [93]:
vect.columns[-30:]

Index(['zushaelinson', 'zust', 'zuwachs', 'zuwanderer', 'zuzu', 'zuzuceo',
       'zvezdanews', 'zvjezdanpatz', 'zw', 'zwangsfinanzierte', 'zwangsr',
       'zweifach', 'zweifaktor', 'zweij', 'zweitwichtigste', 'zwilling',
       'zwillingen', 'zwischendurch', 'zwischenstationen', 'zyka',
       'zynalturist', 'zynischen', 'zyoritv', 'zzcrane', 'zzion', 'zzjwmc',
       'zzzs', 'zzzzzz', 'zzzzzzz', 'zzzzzzzzzzzzzzzzz'],
      dtype='object')

In [91]:
# previously: 94809 columns
# spacy lem: 89679
94809-83709

11100

In [95]:
tweets = df[['text', 'lem']]
tweets

,text,lem
0,#IslamKills Are you trying to say that there w...,islamkill try say terrorist attack europe refu...
1,"Clinton: Trump should’ve apologized more, atta...",clinton trump apologize attack less
2,RT @ltapoll: Who was/is the best president of ...,good president past retweet
3,RT @jww372: I don't have to guess your religio...,guess religion christmasaftermath
4,RT @Shareblue: Pence and his lawyers decided w...,pence lawyer decide official email public coul...
...,...,...
203477,"RT @AndreaChalupa: In intel circles, the story...",intel circles story go fsb film trump orgy rus...
203478,RT @KansasCityDNews: Tonganoxie police: Middle...,tonganoxie police middle school girl sexually ...
203479,RT @signsinyork: Getting the right #company lo...,get right company logo business get message ac...
203480,The Latest: Obama affirms continuity of ties w...,late obama affirm continuity tie canada


In [96]:
# with open('../data_files/tweets.pickle', 'wb') as to_write:
#    pickle.dump(tweets, to_write)

## Word Clouds

### Most frequent tweeters

In [ ]:
fdist = FreqDist(df['user_key'])

wc = WordCloud(width=600, height=400, max_words=50).generate_from_frequencies(fdist)
plt.figure(figsize=(12, 10))
plt.imshow(wc, interpolation='bilinear')
plt.axis('off')
plt.show()

### Most used hashtags in dataset

In [ ]:
fdist = FreqDist(df['hashtags'].apply(lambda x: " ".join(x)))

wc = WordCloud(width=800, height=600, max_words=150).generate_from_frequencies(fdist)
plt.figure(figsize=(12, 10))
plt.imshow(wc, interpolation='bilinear')
plt.axis('off')
plt.show()

Merkelmussbleiben = 'Merkel must stay'

## NLP Pipeline

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem import PorterStemmer

In [ ]:
class NLPPipe:
    
    def __init__(self, vectorizer=CountVectorizer(), cleaning_function=None, tokenizer=None, stemmer=None):
        '''
        Create a pipeline that vectorizes an arbitary list of documents.
        '''
        self.vectorizer = vectorizer
        self.cleaning_function = cleaning_function
        self.tokenizer = tokenizer
        self.stemmer = stemmer
     
    def clean_text(self, text):
        text = text.astype(str)
        cleaned_text = text.map(self.cleaning_function)
        return cleaned_text
    
    def fit(self, text):
        pass
        
    def transform(self, text):
        pass

In [ ]:
nlp = NLPPipe(vectorizer=CountVectorizer(), 
              cleaning_function=clean_tweet, 
              tokenizer=TreebankWordTokenizer().tokenize, 
              stemmer=PorterStemmer())

In [ ]:
df2 = raw_df[['text']]

tweets = df2['text']

df2.head()

In [ ]:
clean_tweets = nlp.clean_text(tweets)

In [ ]:
df2.head()

In [ ]:
for tweet in clean_tweets[0:5]:
    print(tweet, '\n')

In [ ]:
for tweet in raw_df.text[0:5]:
    print(tweet, '\n')

In [ ]:
# df.to_csv('data_files/clean_tweets.zip', index=False)